In [1]:
import os
import sys
import pandas as pd
import numpy as np
import napari
from skimage.io import imread, imsave
import pickle

In [10]:
# Append the path of the scripts directory to sys.path
sys.path.append('./src/statistics_collection/')

In [12]:
from misc import *
from StatsCollector import StatsCollector
from LabelPreprocessing import process_labels, get_labels_touching_edges
from GenMeshes import convert_labels_to_meshes

In [13]:
os.getcwd()

'/nas/groups/iber/Users/Federico_Carrara/Statistics_Collection/EpiStats'

In [14]:
from src.tests.CubeLatticeTest import *

labeled_img = generate_cube_lattice_image(
    nb_x_voxels=10,
    nb_y_voxels=10,
    nb_z_voxels=10,
    cube_side_length=3,
    nb_cubes_x=3,
    nb_cubes_y=3,
    nb_cubes_z=3,
    interstitial_space=-1,
)

In [45]:
viewer = napari.Viewer()
viewer.add_labels(labeled_img)

<Labels layer 'labeled_img' at 0x7ff0cdc5a6b0>

In [ ]:
smoothing_iters = 10
erosion_iters = 6
dilation_iters = 8

PATH_TO_OUTPUT = 'src/tests/cube_output'
output_dir = create_output_directory(
    output_folder=PATH_TO_OUTPUT,
    smoothing_iterations=smoothing_iters, 
    erosion_iterations=erosion_iters, 
    dilation_iterations=8
)

preprocessed_labeled_img = process_labels(
    labeled_img=labeled_img, 
    erosion_iterations=erosion_iters,
    dilation_iterations=dilation_iters,
    output_directory=output_dir,
    overwrite=False
)

meshes = convert_labels_to_meshes(
    preprocessed_labeled_img,
    [1, 1, 1],
    10,
    output_dir,
    False,
    10,
    'stl'
)

In [47]:
stats_collector = StatsCollector(
    meshes=meshes,
    labels=preprocessed_labeled_img,
    features=['area', 'volume', 'elongation_and_axes', 'neighbors', 'contact_area'],
    output_directory=output_dir,
    path_to_img=os.path.join(output_dir, 'processed_labels.tif'),
    tissue='lung_bronchiole',
    num_workers=2
)

Checking labels touching edges: 100%|██████████| 27/27 [00:00<00:00, 114390.11it/s]


In [48]:
stats_collector.collect_statistics()

Computing cell neighbors: 100%|██████████| 27/27 [00:00<00:00, 187.92it/s]


Compute cell contact area ...
Skipping cell 1...Skipping cell 2...

Skipping cell 3...
Skipping cell 4...
Skipping cell 5...
Skipping cell 6...
Skipping cell 7...
Skipping cell 8...
Skipping cell 9...
Skipping cell 10...
Skipping cell 11...
Skipping cell 12...
Skipping cell 13...
Computing contact area for cell 14... 
Computing contact area for cell 15... 
Skipping cell 16...
Computing contact area for cell 17... 
Computing contact area for cell 18... 
Skipping cell 19...
Skipping cell 20...
Skipping cell 21...
Skipping cell 22...
Computing contact area for cell 23... 
Computing contact area for cell 24... 
Skipping cell 25...
Computing contact area for cell 26... 
Computing contact area for cell 27... 


## Test filtering

In [4]:
img = imread('output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
idxs = get_labels_touching_edges(labeled_img=img)

Checking labels touching edges: 100%|██████████| 322/322 [00:00<00:00, 540658.88it/s]


In [7]:
v = napari.Viewer()
v.add_labels(img)
for id in idxs:
    single_cell = img.copy()
    single_cell[single_cell != id] = 0
    v.add_labels(single_cell, name=f'cell_{id}')

## Test 2D stats collection

In [56]:
v = napari.Viewer()

In [63]:
from src.tests.CubeLatticeTest import generate_cube_lattice_image

test_cubes = generate_cube_lattice_image(20, 20, 20, 5, 2, 2, 2, 0)
v.add_labels(test_cubes)
test_cubes.shape

(20, 20, 20)

In [119]:
from StatsUtils import compute_2D_statistics

feat_dicts = compute_2D_statistics(labeled_img=test_cubes, slicing_dim=0, exclude_labels=[], pixel_size=[1, 1])

Computing statistics by slice: 100%|██████████| 20/20 [00:00<00:00, 3889.92it/s]


In [121]:
import pickle

save_name = f'try_save.pickle'
with open(os.path.join('./images/', save_name), 'wb') as file:
    pickle.dump(feat_dicts, file)

In [138]:
img = imread('output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
neighbors, areas = compute_2D_statistics(labeled_img=img, slicing_dim=0, exclude_labels=[], pixel_size=[0.1625, 0.1625])

Computing statistics by slice: 100%|██████████| 1087/1087 [00:39<00:00, 27.70it/s] 


In [150]:
from StatsCollector import StatsCollector

neigh = StatsCollector._unpack_feature_dict(neighbors)
ar = StatsCollector._unpack_feature_dict(areas)

## Test StatsCollector on cubes

In [187]:
from src.tests.CubeLatticeTest import generate_cube_lattice_image

test_cubes = generate_cube_lattice_image(20, 20, 20, 5, 3, 3, 3, 0)
imsave('./src/tests/cube_lattice1.tif', test_cubes)

In [10]:
from run import main

main('./config_test.json')

Preprocessing labels...


AttributeError: 'str' object has no attribute 'copy'

In [196]:
cube_df = pd.read_csv('output_cube_lattice1_s_10_e_6_d_8/cell_stats/stats_dataset_embryo.csv', index_col=0)
cube_df

,cell_ID,tissue,tissue_type,file_name,mesh_dir,exclude_cell,area,volume,neighbors,neighbors_2D,area_2D,isoperimetric_ratio,num_neighbors,elongation,principal_axes,contact_area_fraction,contact_area_distribution,mean_contact_area,total_contact_area,num_neighbors_2D
0,1,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943926,125,[ 2 4 5 10 11 13],"[array([ 2, 10, 11], dtype=uint16), array([ 2,...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458370,6,1.001908,[ 0.57735097 0.57734957 -0.57735027],0.0,[0. 0. 0. 0. 0. 0.],0.0,0.0,"[3, 3, 3, 3, 3]"
1,2,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943927,125,[ 1 3 4 5 6 10 11 12 14],"[array([ 1, 3, 10, 11, 12], dtype=uint16), ar...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458375,9,1.001908,[-0.57735079 -0.57734916 0.57735087],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[5, 5, 5, 5, 5]"
2,3,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943928,125,[ 2 5 6 11 12 15],"[array([ 2, 11, 12], dtype=uint16), array([ 2,...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458383,6,1.001908,[-0.5773508 -0.57734921 0.5773508 ],0.0,[0. 0. 0. 0. 0. 0.],0.0,0.0,"[3, 3, 3, 3, 3]"
3,4,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943927,125,[ 1 2 5 7 8 10 13 14 16],"[array([ 1, 2, 11, 19, 20], dtype=uint16), ar...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458375,9,1.001908,[ 0.57735055 0.57735017 -0.57735009],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[5, 5, 5, 5, 5]"
4,5,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943928,125,[ 1 2 3 4 6 7 8 9 11 13 14 15 17],"[array([ 1, 2, 3, 10, 12, 19, 20, 21], dtype...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458379,13,1.001908,[-0.57735037 -0.57734975 0.57735068],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[8, 8, 8, 8, 8]"
5,6,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943929,125,[ 2 3 5 8 9 12 14 15 18],"[array([ 2, 3, 11, 20, 21], dtype=uint16), ar...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458387,9,1.001908,[ 0.57735038 0.57734981 -0.57735062],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[5, 5, 5, 5, 5]"
6,7,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943928,125,[ 4 5 8 13 16 17],"[array([10, 11, 20], dtype=uint16), array([10,...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458383,6,1.001908,[ 0.57735061 0.5773501 -0.5773501 ],0.0,[0. 0. 0. 0. 0. 0.],0.0,0.0,"[3, 3, 3, 3, 3]"
7,8,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943929,125,[ 4 5 6 7 9 14 16 17 18],"[array([10, 11, 12, 19, 21], dtype=uint16), ar...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458387,9,1.001908,[-0.57735043 -0.57734969 0.5773507 ],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[5, 5, 5, 5, 5]"
8,9,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943931,125,[ 5 6 8 15 17 18],"[array([11, 12, 20], dtype=uint16), array([11,...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458395,6,1.001908,[-0.57735044 -0.57734974 0.57735063],0.0,[0. 0. 0. 0. 0. 0.],0.0,0.0,"[3, 3, 3, 3, 3]"
9,10,embryo,Undefined,./output_cube_lattice1_s_10_e_6_d_8/processed_...,./output_cube_lattice1_s_10_e_6_d_8/cell_meshe...,False,165.943927,125,[ 1 2 4 11 13 14 19 20 22],"[array([ 5, 13, 14], dtype=uint16), array([ 5,...","[25.0, 25.0, 25.0, 25.0, 25.0]",292.458375,9,1.001908,[-0.57735079 -0.57735017 0.57734986],0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],0.0,0.0,"[3, 3, 3, 3, 3]"


### Test on Embrios data

In [19]:
# labeled_img = imread('/nas/groups/iber/Projects/Early_Embryo/maoxia/movie_4/curation/cell_boundary_time_1.tif')

# viewer = napari.Viewer()
# viewer.add_labels(labeled_img)

In [40]:
from run import main

main('./src/statistics_collection/config_test.json')

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
Preprocessing labels...


Applying erosion and dilation: 100%|██████████| 9/9 [00:04<00:00,  1.97it/s]


Computing cell neighbors: 100%|██████████| 8/8 [00:03<00:00,  2.64it/s]

Computing cell contact area ...

    Calculating contact area for cell 1/8 ... 
    Calculating contact area for cell 2/8 ... 
    Calculating contact area for cell 3/8 ... 
    Calculating contact area for cell 4/8 ... 
    Calculating contact area for cell 5/8 ... 
    Calculating contact area for cell 6/8 ... 
    Calculating contact area for cell 7/8 ... 
    Calculating contact area for cell 8/8 ... 


Computing cell 2D statistics: 100%|██████████| 512/512 [00:01<00:00, 372.52it/s] 


In [24]:
embryo_df = pd.read_csv('outputs/output_cell_boundary_time_1_s_10_e_6_d_8/cell_stats/stats_dataset_embryo.csv', index_col=0)
embryo_df

,cell_ID,tissue,tissue_type,file_name,mesh_dir,exclude_cell,area,volume,neighbors,neighbors_2D,area_2D,isoperimetric_ratio,num_neighbors,elongation,principal_axes,contact_area_fraction,contact_area_distribution,mean_contact_area,total_contact_area,num_neighbors_2D
0,1,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,79903.401032,1720993,[3 4 5 7],"[[], [], [], [], [], [], [], [], [], [], [], [...","[208.0, 473.0, 734.0, 1001.0, 1283.0, 1560.0, ...",172.241416,4,1.345604,[-0.72548869 0.68812495 0.01225603],0.251677,[9013.89771219 6097.64018001 3947.05451218 216...,5306.137183,21224.548732,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,45111.919440,625132,[3 5 6],"[[3, 6], [3, 6], [3, 6], [3, 6], [3, 6], [3, 5...","[10.0, 44.0, 85.0, 136.0, 185.0, 259.0, 346.0,...",234.925661,3,1.584644,[-0.0776917 0.99692296 0.01042176],0.227088,[4015.22721885 3047.99892525 3749.96768263],3604.397942,10813.193827,"[2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
2,3,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,44050.517437,476838,[1 2 4 5 6],"[[4], [4], [1, 4], [1, 4], [1, 4], [1, 4], [1,...","[4.0, 69.0, 226.0, 399.0, 527.0, 632.0, 713.0,...",375.933759,5,1.503350,[ 0.00980229 -0.99929902 0.03613 ],0.432150,[9056.33588047 4012.1008636 2443.32289006 374...,4058.940327,20294.701637,"[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,4,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,37756.731436,321880,[1 3 5 6 7],"[[], [], [], [], [], [7], [7], [7], [7], [7], ...","[47.0, 109.0, 228.0, 383.0, 534.0, 654.0, 737....",519.511526,5,1.379881,[0.33057493 0.94371026 0.0114527 ],0.546649,[6134.11089683 2435.97559058 2755.5988418 277...,4480.104313,22400.521567,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,44050.989645,651925,[1 2 3 4 6 7],"[[7], [7], [1, 4, 7], [1, 4, 7], [1, 4, 7], [1...","[1.0, 11.0, 64.0, 195.0, 327.0, 481.0, 672.0, ...",201.127631,6,1.352725,[-0.64907215 0.75687748 -0.07643178],0.561673,[ 3928.06979293 3041.23276467 3746.72542953 ...,4471.182662,26827.095974,"[1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
5,6,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,45256.378226,602208,[2 3 4 5 7],"[[7], [4, 7], [4, 7], [4, 7], [4, 7], [4, 7], ...","[8.0, 46.0, 175.0, 279.0, 396.0, 526.0, 672.0,...",255.591464,5,1.379023,[-0.64693752 0.76127133 -0.04402053],0.441529,[ 3716.52268663 1056.91017935 2750.43304937 ...,4281.426651,21407.133256,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
6,7,embryo,Undefined,./outputs/output_cell_boundary_time_1_s_10_e_6...,./outputs/output_cell_boundary_time_1_s_10_e_6...,False,42325.048455,648437,[1 4 5 6],"[[], [], [], [], [], [], [], [], [], [], [], [...","[44.0, 199.0, 391.0, 602.0, 795.0, 980.0, 1164...",180.325363,4,1.401635,[0.64332036 0.76557831 0.00536416],0.381736,[2205.73238182 8237.3297426 3129.54172204 366...,4309.338152,17237.352608,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [38]:
def chiama_mamma(name):
    print(f'Ciao mamma {name}.')

def auguri_mamma(age):
    print(f'Auguri mamma per i tuoi {age} anni.')

def saluta(methods, *args, **kwargs):
    if 'chiama' in methods:
        chiama_mamma(kwargs.pop('name'))
    if 'auguri' in methods:
        auguri_mamma(kwargs.pop('age'))

In [39]:
saluta(['chiama', 'auguri'], name='marina', age=57)

Ciao mamma marina.
Auguri mamma per i tuoi 57 anni.


## Test contact area computation

In [ ]:
import napari

viewer = napari.Viewer()
img = imread('output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
viewer.add_labels(img, name='Original image')

In [45]:
from StatsUtils import _compute_contact_area
import trimesh as tm

# Upload required data
path_to_meshes = 'output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/cell_meshes'
mesh_files = sorted(os.listdir(path_to_meshes), key=lambda x: int(x.split('_')[1].split('.')[0]))
mesh_files

meshes_dict = {}
for i, mesh_file in enumerate(mesh_files):
    meshes_dict[i+1] = tm.load_mesh(os.path.join(path_to_meshes, mesh_file))

with open(os.path.join('output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/cell_stats/cached_stats/cell_neighbors.pickle'), 'rb') as file:
    neighbors_dict = pickle.load(file)

In [46]:
cell_id = 5
neighbors_list = neighbors_dict[cell_id]
neighbors_list

array([ 7, 22, 52, 59], dtype=uint32)

In [47]:
results = _compute_contact_area(meshes_dict, cell_id, neighbors_list, 0.65, show_napari=True)

    Calculating contact area for cell 5/322 ... 


In [50]:
results[0], np.sum(results[1]), meshes_dict[cell_id].area

(0.7293887804749181, 453.65012084051455, 505.85460734373214)

In [8]:
viewer = napari.Viewer()
img = imread('output_cell_boundary_time_1_s_10_e_6_d_8/processed_labels.tif')
viewer.add_labels(img, name='Original image')
for id, mesh in meshes_dict.items():
    viewer.add_surface((mesh.vertices, mesh.faces), name=f'Mesh_{id}', blending='additive', opacity=0.6)